In [12]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.nn as nn  
import torch.optim as optim



In [13]:
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.features = self.data.iloc[:, :-1].values
        self.targets = self.data.iloc[:, -1].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)  
        return features, target

csv_file = "./MEFAR_DOWN_WITHOUT.csv"
dataset = CustomDataset(csv_file)

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
test_loader = DataLoader(test_set, batch_size=10, shuffle=False)

In [14]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_dim = 17
        self.out_dim = 1
        self.fc1 = nn.Linear(self.in_dim, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)
        self.fc5 = nn.Linear(2, self.out_dim)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        a1 = self.relu(self.fc1(x))
        a2 = self.relu(self.fc2(a1))
        a3 = self.relu(self.fc3(a2))
        a4 = self.relu(self.fc4(a3))
        output = self.fc5(a4)  # Pas besoin de log_softmax pour la régression
        return output

In [15]:
model = MLP()
criterion = nn.MSELoss()  # Utilisation de MSELoss pour la régression
optimizer = optim.SGD(model.parameters(), lr=0.00001)  

In [16]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)  # Squeeze pour éliminer les dimensions inutiles
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if (i + 1) % 2000 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: nan
[2,  2000] loss: nan
[3,  2000] loss: nan
[4,  2000] loss: nan
[5,  2000] loss: nan
[6,  2000] loss: nan
[7,  2000] loss: nan
[8,  2000] loss: nan
[9,  2000] loss: nan
[10,  2000] loss: nan
Finished Training
